In [144]:
################################################################
#This file will summarize data from the "Heros of Pamoli" game#
###############################################################

In [166]:
#import dependencies
import pandas as pd
import numpy as np

In [167]:
#read the master data file and create a dataframe
masterFile = "Resources/purchase_data.csv"
master_df = pd.read_csv(masterFile)
master_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [147]:
#create the age group category for later and add to dataframe, completing my master dataframe
bins=[0,9,14,19,24,29,34,39,400]
labels=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
master_df["Age Group"] = pd.cut(master_df["Age"], bins, labels=labels, include_lowest=True)

In [148]:
#Create a dataframe to display total number of players
unique_players=master_df["SN"].unique()
total_players=np.count_nonzero(unique_players)
total_players_df =pd.DataFrame([
    {"Total Players":total_players}])
total_players_df

,Total Players
0,576


In [149]:
#Create a dataframe to display basic info about purchases
unique_items=master_df["Item Name"].unique()
total_unique_items=np.count_nonzero(unique_items)
average_price=master_df["Price"].mean()
total_purchases=master_df["Purchase ID"].count()
total_revenue=master_df["Price"].sum()

basic_info = pd.DataFrame([
        {"Number of Unique Items": total_unique_items,
         "Average Price":average_price,
         "Number of Purchases": total_purchases,
         "Total Revenue": total_revenue}
    ])
basic_info

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [150]:
#create a dataframe to display the gender breakdown of players
unique_sn_df = master_df.drop_duplicates(subset='SN', keep="first")
male_players = unique_sn_df.loc[master_df["Gender"] == "Male", "Gender"]
female_players = unique_sn_df.loc[master_df["Gender"] == "Female", "Gender"]
other_players = unique_sn_df.loc[master_df["Gender"] == "Other / Non-Disclosed", "Gender"]

playerCount_df = pd.DataFrame({ 
    "Gender": ["Female", "Male", "Other"],
    "Total Count": [female_players.count(), male_players.count(), other_players.count()],
   })

percent_gender = playerCount_df["Total Count"]/total_players*100
playerCount_df["Percentage of Players"] = percent_gender

playerCount_df

,Gender,Total Count,Percentage of Players
0,Female,81,14.062500
1,Male,484,84.027778
2,Other,11,1.909722


In [151]:
#Create a table of summary statistics concerning Male and Female purchases

# Use GroupBy to get most of our Gender data
grouped_gender_df = master_df.groupby(['Gender'])

#Get our data functions from grouped gender data
gender_priceMean=grouped_gender_df["Price"].mean()
gender_purchaseCount=grouped_gender_df["Purchase ID"].count()
gender_totalvalue=grouped_gender_df["Price"].sum()

#### Get the average total purchase per person. I know there's an easier way to do this, but I haven't found it yet.
#Get female average spent per person
female_df=master_df.loc[master_df["Gender"]=="Female", :] #dataset of only females
grouped_gender_sn_df = female_df.groupby(['SN'])
F_total_purchase_per=grouped_gender_sn_df["Price"].sum()
F_total_purchase_per.mean()

#Get male average spent per person
male_df=master_df.loc[master_df["Gender"]=="Male", :] #dataset of only Males
grouped_gender_sn_df = male_df.groupby(['SN'])
M_total_purchase_per=grouped_gender_sn_df["Price"].sum()
M_total_purchase_per.mean()

#Get other gender average spent per person
other_df=master_df.loc[master_df["Gender"]=="Other / Non-Disclosed", :] #dataset of only Others
grouped_gender_sn_df = other_df.groupby(['SN'])
O_total_purchase_per=grouped_gender_sn_df["Price"].sum()
O_total_purchase_per.mean()

#Put these in a list
avg_total_list=[F_total_purchase_per.mean(), M_total_purchase_per.mean(),O_total_purchase_per.mean()]
####

#Put everything in a dataframe
gender_stats_df = pd.DataFrame({"Purchase Count": gender_purchaseCount,
                                "Average Purchase Price": gender_priceMean,
                                "Total Purchase Value": gender_totalvalue,
                               "Average Total Purchase per Person": avg_total_list})

gender_stats_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


In [152]:
#Create a table of summary statistics for players of different age ranges

master_df["Age Group"] = pd.cut(master_df["Age"], bins, labels=labels, include_lowest=True)
age_group_df=unique_sn_df.groupby("Age Group")
age_count=age_group_df["Purchase ID"].count()
age_percent =age_count/total_players
age_df = pd.DataFrame({"Total Count":age_count,
                      "Percentage of Players":age_percent})
age_df

,Total Count,Percentage of Players
Age Group,,
<10,17,0.029514
10-14,22,0.038194
15-19,107,0.185764
20-24,258,0.447917
25-29,77,0.133681
30-34,52,0.090278
35-39,31,0.053819
40+,12,0.020833


In [171]:
#Create Chart of Top Spenders

master_df["count"]=1
grouped_SN=master_df.groupby(["SN"])
countsum=grouped_SN.sum()
sorted_countsum=countsum.sort_values(["Price","SN"], ascending=False)
price=sorted_countsum["Price"]
count=sorted_countsum["count"]
sorted_countsum["Average Purchase Price"]=price/count
sorted_countsum["Total Purcase Value"]=price*count
top_spenders = sorted_countsum.reset_index(drop=False)
top_spenders = top_spenders.rename(columns={"count":"Purchase Count"})
top_spenders_cleaned=top_spenders.drop(['Item ID','Purchase ID', 'Age','Price'], axis=1)
top_spenders_cleaned.head()

,SN,Purchase Count,Average Purchase Price,Total Purcase Value
0,Lisosia93,5,3.792000,94.80
1,Idastidru52,4,3.862500,61.80
2,Chamjask73,3,4.610000,41.49
3,Iral74,4,3.405000,54.48
4,Iskadarya95,3,4.366667,39.30


In [ ]:
#Most Popular Items

In [172]:
master_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,count
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,1
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,1
2,2,Ithergue48,24,Male,92,Final Critic,4.88,1
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,1
4,4,Iskosia90,23,Male,131,Fury,1.44,1


In [194]:
MP_sorted=master_df.sort_values(["count"], ascending=True)

In [196]:
MP_grouped=master_df(["Item ID", "Item Name"])

TypeError: 'DataFrame' object is not callable

In [193]:
MP_grouped.sum()

,,Purchase ID,Age,Price,count
Item ID,Item Name,,,,
0,Splinter,1149,74,5.12,4
1,Crucifer,1523,89,11.77,4
2,Verdict,2006,143,14.88,6
3,Phantomlight,2087,155,14.94,6
4,Bloodlord's Fetish,1885,124,8.50,5
...,...,...,...,...,...
178,"Oathbreaker, Last Hope of the Breaking Storm",4960,287,50.76,12
179,"Wolf, Promise of the Moonwalker",1948,126,26.88,6
181,Reaper's Toll,1278,109,8.30,5
